In [9]:
pip install folium pandas mcdm

   ---------------------------------------- 0.0/99.1 kB ? eta -:--:--
   ------------ --------------------------- 30.7/99.1 kB 1.4 MB/s eta 0:00:01
   --------------------------------- ------ 81.9/99.1 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 99.1/99.1 kB 1.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# 예시 데이터 (대여소 위치 및 관련 정보)
data = {
    '대여소명': ['망원역 1번출구 앞', '합정역 2번출구 앞', '홍대입구역 3번출구 앞'],
    '위도': [37.556, 37.550, 37.558],
    '경도': [126.910, 126.914, 126.923],
    '유동인구': [1000, 1500, 1200],
    '이용량': [500, 800, 600]
}

df = pd.DataFrame(data)

In [13]:
# 지도 생성 (서울 중심 좌표 설정)
m = folium.Map(location=[37.550, 126.920], zoom_start=13)

# 마커 클러스터 생성
marker_cluster = MarkerCluster().add_to(m)

# 대여소 위치에 마커 추가
for idx, row in df.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=f"{row['대여소명']}\n유동인구: {row['유동인구']}\n이용량: {row['이용량']}",
        icon=folium.Icon(color='blue')
    ).add_to(marker_cluster)

# 지도 출력
m.save('bikeshare_map.html')
m

In [19]:
import mcdm

# 의사결정 행렬 생성 (유동인구와 이용량이 기준)
decision_matrix = df[['유동인구', '이용량']].values

# Min-Max 정규화 수행 (유동인구와 이용량을 각각 정규화)
normalized_matrix = (decision_matrix - decision_matrix.min(axis=0)) / (decision_matrix.max(axis=0) - decision_matrix.min(axis=0))

# 가중치 설정 (예: 유동인구와 이용량의 중요도를 각각 0.6과 0.4로 설정)
weights = [0.6, 0.4]

# SAW(Simple Additive Weighting) 방법으로 순위 계산
rankings = mcdm.rank(normalized_matrix, w_vector=weights, s_method="SAW")

# 결과 출력 (순위가 높은 대여소가 최적 입지)
df['순위'] = rankings
print(df[['대여소명', '순위']])

           대여소명                         순위
0    망원역 1번출구 앞                  (a2, 1.0)
1    합정역 2번출구 앞  (a3, 0.37333333333333335)
2  홍대입구역 3번출구 앞                  (a1, 0.0)


In [21]:
# 순위에 따라 마커 색상 변경
for idx, row in df.iterrows():
    color = 'green' if row['순위'] == max(df['순위']) else 'blue'
    
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=f"{row['대여소명']}\n순위: {row['순위']}",
        icon=folium.Icon(color=color)
    ).add_to(marker_cluster)

# 업데이트된 지도 출력
m.save('bikeshare_map_ranked.html')
m